<a href="https://colab.research.google.com/github/jesung/Tic-Tac-Toe/blob/master/tic_tac_toe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pprint
import numpy as np
import tensorflow as tf
from tensorflow import keras
import random

model = keras.models.Sequential([
    keras.layers.Dense(18, activation=tf.nn.relu, input_shape=(1,9)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(9, activation=tf.nn.relu),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(1, activation=tf.nn.sigmoid)
])

model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy','binary_crossentropy'])

class tic_tac_toe:
    def __init__(self):
        self.board = np.array([[2,2,2],[2,2,2],[2,2,2]])
        self.turn = 0

    def reset(self):
        self.board = np.array([[2,2,2],[2,2,2],[2,2,2]])
        self.turn = 0

    def move(self,x,y):
        ##### need to update #######
        if self.board[x,y] != 2:   
            print('Invalid move',x,y)
            print(self.board)
        else:
            self.board[x,y] = self.turn
            
            if self.turn == 0:
                self.turn = 1
            else:
                self.turn = 0

    def win(self):
        for i in range(3):
            if self.board[0,i]==self.board[1,i]==self.board[2,i]<2:
                return 'Won'
            if self.board[i,0]==self.board[i,1]==self.board[i,2]<2:
                return 'Won'

        if self.board[0,0]==self.board[1,1]==self.board[2,2]<2:
            return 'Won'
        if self.board[0,2]==self.board[1,1]==self.board[2,0]<2:
            return 'Won'

        if not 2 in self.board:
            return 'Drawn'
        else:
            return 'In-Progress'

def move_gen(board_state, turn):
    legal_moves = {}
    
    for i in range(3):
        for j in range(3):
            if board_state[i,j] == 2:
                tmp_board = board_state.copy()
                tmp_board[i,j] = turn
                legal_moves[(i,j)]=tmp_board.flatten()

    return legal_moves
  
def move_sel(move_list, turn, train=True):
    tmp = np.zeros((9,12))
    i = 0
    if train:
      k = 0
    else:
      k = 10

    for key, value in move_list.items():
      tmp[i,0:9] = value.reshape(1,9)
      tmp[i,9] = model.predict(value.reshape(1,1,9))[0][0][0]
      tmp[i,10:12] = [key[0], key[1]]
      i = i + 1
    
    if random.randint(0,5) == k:
      if turn == 0:
        j = random.choices(range(i),tmp[:i,9]**2)
      else:  
        j = random.choices(range(i),(1-tmp[:i,9])**2)
    else:
      if turn == 0:
        j = np.argmax(tmp[:i,9])
      else:
        j = np.argmin(tmp[:i,9])
    
    #return tmp[j,0:9], tmp[j,9][0], (tmp[j,10][0],tmp[j,11][0])
    return tmp[j,0:9], tmp[j,9], (tmp[j,10],tmp[j,11])

def simulate(batch_size, num_epoch):
  for i in range(num_epoch):
    board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
    train = [np.append(board,prob)]
    
    for j in range(batch_size):
      #clean up - train & tmp outside for loops and clear at beginning
      board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
      tmp = [np.append(board,prob)]

      while game.win() == 'In-Progress':
        board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
        tmp = np.append(tmp, [np.append(board,prob)], axis = 0)
        
        #print(move[0], move[1])
        game.move(int(move[0]), int(move[1]))

      #update score based on result
      if game.win() == 'Won':
        tmp[:,-1] = game.turn
      else:
        tmp[:,-1] = 0.5
      
      #add to training set
      train = np.append(train, tmp, axis = 0)
      
      game.reset()
    
    #print(train.shape)
    #print(train)
    print('Epoch', i)
    model.fit(train[:,0:9].reshape(-1,1,9), train[:,9].reshape(-1,1,1),epochs=5)

In [0]:
game = tic_tac_toe()

simulate(500,50)



Epoch 0
Epoch 1/5
4704/4704 [==============================] - 1s 276us/step - loss: 0.7220 - acc: 0.1603 - binary_crossentropy: 0.7220
Epoch 2/5
4704/4704 [==============================] - 0s 47us/step - loss: 0.6936 - acc: 0.1711 - binary_crossentropy: 0.6936
Epoch 3/5
4704/4704 [==============================] - 0s 46us/step - loss: 0.6894 - acc: 0.1745 - binary_crossentropy: 0.6894
Epoch 4/5
4704/4704 [==============================] - 0s 46us/step - loss: 0.6870 - acc: 0.1728 - binary_crossentropy: 0.6870
Epoch 5/5
4704/4704 [==============================] - 0s 46us/step - loss: 0.6855 - acc: 0.1803 - binary_crossentropy: 0.6855
Epoch 1
Epoch 1/5
4373/4373 [==============================] - 0s 46us/step - loss: 0.5030 - acc: 0.6769 - binary_crossentropy: 0.5030
Epoch 2/5
4373/4373 [==============================] - 0s 45us/step - loss: 0.4835 - acc: 0.6769 - binary_crossentropy: 0.4835
Epoch 3/5
4373/4373 [==============================] - 0s 51us/step - loss: 0.4798 - acc: 0.67

In [70]:
game.reset()
#game.move(0,0)
print(game.win())
print(game.board)

print(move_sel(move_gen(game.board, game.turn), game.turn, train=False))


Won
[[1 2 0]
 [2 0 2]
 [1 1 0]]
(array([1., 2., 0., 0., 0., 2., 1., 1., 0.]), 0.9974275231361389, (1.0, 0.0))


In [0]:
game.reset()
board, prob, move = move_sel(move_gen(game.board, game.turn), game.turn)
tmp = [np.append(board,prob)]
